In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import json

with open('funbigfile.json') as f:
    data = json.load(f)

df = pd.json_normalize(data['bruh'])
df = df.drop(['type', 'uri', 'track_href', 'analysis_url','time_signature', 'duration_ms'], axis=1)
#print(df)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('genre', axis=1), df['genre'], test_size=0.2, random_state=42)

# Encode the target variable
lb = LabelBinarizer()
y_train_encoded = lb.fit_transform(y_train)
y_test_encoded = lb.transform(y_test)

# Define the features to be used for training
#features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']
#features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'speechiness', 'tempo', 'valence']
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness','speechiness', 'tempo', 'valence']
# Create the feature matrix
X_train = X_train[features]
X_test = X_test[features]

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train_encoded)

#debugging stuff:
#print("Testing dataset:")
#print(y_test)
#more debugging stuff 
y_pred = rf.predict(X_test[features])
#print("Predicted genres:")
#print(type(lb.inverse_transform(y_pred)))
#print(lb.inverse_transform(y_pred))


# Evaluate the performance of the model on the test data
accuracy = rf.score(X_test, y_test_encoded)
print("Accuracy:", accuracy)

print("TESTING DATASET: ")
print(y_test.values)

print("GUESSES: ")
print(lb.inverse_transform(y_pred))

rightvals = y_test.values
guessedvals = lb.inverse_transform(y_pred)

incorrect_guesses = rightvals != guessedvals
right_unguessed = rightvals[incorrect_guesses]
incorrect_preds = guessedvals[incorrect_guesses]
print("INCORRECT GUESSES: ")
#print(right_unguessed) #these are the things that were NOT guessed for
#print(incorrect_preds) #these are the things we guessed instead

comparearr = []
for i in range(0,len(right_unguessed),1):
    comparearr.append("CORRECT:" + right_unguessed[i] + " OUR GUESS:" + incorrect_preds[i])

print(comparearr)


#phonk
#np.count_nonzero(y == 1)
genreList = list(set(rightvals))

for g in genreList:
    print('\n')
    total = np.count_nonzero(rightvals == g)
    missed = np.count_nonzero(right_unguessed == g)
    prop = str(1-(missed/total))
    print("Proportion of " + g + " guessed correctly: " + prop)
    wrongguess = np.count_nonzero(incorrect_preds == g)
    newprop = str((wrongguess/total))
    print("Proportion of guesses we shouldn't have guessed as " + g + ": " + newprop)